**Setup and Installation**

In [ ]:
!pip install langchain langchain-openai tiktoken pillow matplotlib pandas

import os
import re
import json
import base64
from io import BytesIO
from typing import List, Dict, Any, Optional, Union

os.environ["OPENAI_API_KEY"] = "your-api-key"

import tiktoken
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_openai import OpenAI, ChatOpenAI

**Basic Utility Functions**

In [ ]:
def count_tokens(text: str, model: str = "gpt-3.5-turbo") -> int:
    """Count the number of tokens in a text string."""
    encoder = tiktoken.encoding_for_model(model)
    return len(encoder.encode(text))

def print_separator():
    """Print a visual separator."""
    print("\n" + "="*50 + "\n")

# Create sample text documents for testing
sample_docs = [
    Document(page_content="The Milky Way galaxy contains between 100-400 billion stars and has a diameter of about 100,000 light-years. Our solar system is located in one of the spiral arms, about 27,000 light-years from the galactic center.",
             metadata={"source": "astronomy_textbook", "page": 42}),
    Document(page_content="Black holes are regions of spacetime where gravitational forces are so strong that nothing, not even light, can escape. The boundary of no escape is called the event horizon. Supermassive black holes are found at the center of most galaxies, including our Milky Way.",
             metadata={"source": "astrophysics_journal", "page": 128}),
    Document(page_content="Stars are formed in nebulae, which are clouds of gas and dust. When a nebula's core reaches a critical density and temperature, nuclear fusion begins, and a star is born. The color of a star depends on its temperature, with red stars being cooler and blue stars being hotter.",
             metadata={"source": "stellar_formation_guide", "page": 75}),
    Document(page_content="Exoplanets are planets outside our solar system. As of 2023, over 5,000 exoplanets have been confirmed. They are detected using various methods, including transit photometry and radial velocity measurements. Some exoplanets exist in the 'habitable zone' where conditions might allow for liquid water.",
             metadata={"source": "exoplanet_database", "page": 12}),
]

# Create a multimodal dataset with text and image descriptions
multimodal_data = [
    {
        "text": sample_docs[0].page_content,
        "image_description": "Spiral galaxy viewed from above showing the central bulge and spiral arms with stars, gas, and dust. A small indicator shows our solar system's position in one of the outer spiral arms.",
        "question": "Where is our solar system located in the Milky Way galaxy?",
        "answer": "Our solar system is located in one of the spiral arms of the Milky Way galaxy, approximately 27,000 light-years from the galactic center."
    },
    {
        "text": sample_docs[1].page_content,
        "image_description": "Illustration of a black hole showing the event horizon, accretion disk with bright glowing matter swirling around it, and light bending near the event horizon.",
        "question": "What is an event horizon?",
        "answer": "The event horizon is the boundary of a black hole beyond which nothing, not even light, can escape due to the extremely strong gravitational forces."
    },
    {
        "text": sample_docs[2].page_content,
        "image_description": "Image of a nebula with bright stars forming within clouds of colorful gas and dust. Different colored stars are visible: blue, white, yellow, and red.",
        "question": "What determines the color of a star?",
        "answer": "The color of a star depends on its temperature, with red stars being cooler and blue stars being hotter."
    }
]

# Complex reasoning questions for advanced templates
reasoning_questions = [
    {
        "context": "\n".join([doc.page_content for doc in sample_docs]),
        "question": "Based on the information provided, how might the presence of a supermassive black hole affect star formation in a galaxy?",
        "reasoning": "To answer this question, I need to connect information about black holes and star formation from the context. From the context, I know that supermassive black holes are found at the center of most galaxies, including our Milky Way. I also know that stars form in nebulae when gas and dust reach critical density and temperature. A supermassive black hole could affect star formation in several ways: 1) Its strong gravitational pull could compress nearby gas clouds, potentially triggering star formation; 2) The energy released from matter falling into the black hole could heat surrounding gas, preventing it from cooling and condensing to form stars; 3) Black hole jets and radiation could blow away gas needed for star formation. The context doesn't explicitly state these effects, but connecting the information about black holes and star formation allows for these logical inferences."
    },
    {
        "context": "\n".join([doc.page_content for doc in sample_docs]),
        "question": "How might techniques used to detect exoplanets be limited by the properties of different star types?",
        "reasoning": "This requires connecting information about exoplanet detection methods with star properties. The context mentions that exoplanets are detected using transit photometry and radial velocity measurements, and that stars have different colors/temperatures. For transit photometry, which detects planets when they pass in front of their star, limitations could include: 1) With very bright/hot blue stars, the brightness difference caused by a transit might be harder to detect; 2) With red dwarf stars, which are smaller, the relative size of the planet to the star is larger, making transits more noticeable. For radial velocity, which measures a star's 'wobble' due to a planet's gravity: 1) More massive stars would show less wobble from the same planet; 2) Stars with high activity (like many red dwarfs) might have variations that mask the planetary signal. By connecting the properties of stars with the detection methods, I can infer these limitations."
    }
]

# Generate a sample complex dataset
complex_data = [
    {
        "context": "Mars has two small moons, Phobos and Deimos. Phobos orbits Mars every 7 hours and 39 minutes at an altitude of 6,000 km. It is gradually spiraling inward at a rate of 1.8 cm per year. Deimos orbits Mars every 30 hours at an altitude of 23,460 km. Unlike Phobos, Deimos is gradually moving away from Mars. Both moons are thought to be captured asteroids.",
        "question": "What will eventually happen to Phobos based on its current orbital trajectory?",
        "answer": "Since Phobos is gradually spiraling inward toward Mars at a rate of 1.8 cm per year, it will eventually either crash into Mars or be torn apart by Mars' gravitational forces when it reaches the Roche limit, forming a planetary ring."
    },
    {
        "context": "Europa, a moon of Jupiter, has a surface of solid water ice. Scientists believe there is a liquid water ocean beneath this icy crust, kept warm by tidal heating caused by Jupiter's gravitational pull. The ocean may be up to 100 km deep. Enceladus, a moon of Saturn, also shows evidence of a subsurface ocean, with geysers of water vapor observed erupting from its south pole region.",
        "question": "Compare the mechanisms that might allow for liquid water on Europa and Enceladus.",
        "answer": "Both Europa and Enceladus likely have subsurface liquid water oceans beneath their icy crusts. On Europa, the liquid water is maintained by tidal heating caused by Jupiter's gravitational pull, which prevents the water from freezing despite the cold temperatures far from the Sun. For Enceladus, while not explicitly stated in the context, the presence of geysers erupting from its south pole strongly suggests internal heating, likely also from tidal forces but from Saturn rather than Jupiter. Both moons demonstrate how tidal heating from a massive parent planet can maintain conditions for liquid water far from the Sun's warmth."
    }
]

# Section 1: Multi-modal Prompt Templates
# ----------------------------------------

print("Section 1: Multi-modal Prompt Templates")

# Image-aware RAG Template
multimodal_template = """
Use both the text context and images provided to answer the question.

TEXT CONTEXT:
{text_context}

IMAGE CONTEXT:
{image_descriptions}

QUESTION:
{question}

When answering:
1. Consider information from both text and images
2. If images contain relevant information not in the text, include it
3. Specify whether your information comes from text or images
4. If more visual details would help, mention this

ANSWER:
"""

# Structured Output with Visual References
visual_reference_template = """
Answer the question using the provided text and image information.

TEXT CONTEXT:
{text_context}

IMAGE DESCRIPTIONS:
{image_descriptions}

QUESTION:
{question}

INSTRUCTIONS:
- Provide your answer with references to both text [T1, T2, etc.] and images [I1, I2, etc.]
- If the answer requires visual information not described, say so clearly
- Format tables or structured data appropriately

ANSWER:
"""

# Test multimodal template
example = multimodal_data[0]
formatted_multimodal = multimodal_template.format(
    text_context=example["text"],
    image_descriptions=example["image_description"],
    question=example["question"]
)

print("Multimodal Template Example:")
print(formatted_multimodal)
print(f"Tokens: {count_tokens(formatted_multimodal)}")

# Test visual reference template
formatted_visual_ref = visual_reference_template.format(
    text_context=example["text"],
    image_descriptions=example["image_description"],
    question=example["question"]
)

print("\nVisual Reference Template Example:")
print(formatted_visual_ref)
print(f"Tokens: {count_tokens(formatted_visual_ref)}")

# Example of simulating a multimodal retrieval system
def multimodal_retrieval_simulation(query, corpus):
    """
    Simulate a multimodal retrieval system that returns
    both text and image descriptions.
    In a real system, this would use vector search on embeddings.
    """
    # Simple keyword matching for simulation
    results = []
    for item in corpus:
        if any(word in item["text"].lower() for word in query.lower().split()):
            results.append({
                "text": item["text"],
                "image_description": item["image_description"]
            })

    # Format results for template
    if results:
        text_context = "\n\n".join([r["text"] for r in results])
        image_descriptions = "\n\n".join([f"Image {i+1}: {r['image_description']}"
                                          for i, r in enumerate(results)])
        return {
            "text_context": text_context,
            "image_descriptions": image_descriptions
        }
    else:
        return {
            "text_context": "No relevant text found.",
            "image_descriptions": "No relevant images found."
        }

# Test the multimodal retrieval
query = "black hole event horizon"
retrieved = multimodal_retrieval_simulation(query, multimodal_data)

print("\nMultimodal Retrieval Example:")
print("Query:", query)
print("Retrieved Text:", retrieved["text_context"])
print("Retrieved Images:", retrieved["image_descriptions"])

# Generate a response using the multimodal template
if os.environ.get("OPENAI_API_KEY"):
    try:
        print("\nGenerating response with LLM:")
        llm = ChatOpenAI(temperature=0)
        prompt = multimodal_template.format(
            text_context=retrieved["text_context"],
            image_descriptions=retrieved["image_descriptions"],
            question="What is the significance of a black hole's event horizon?"
        )
        response = llm.invoke(prompt)
        print("Response:", response.content)
    except Exception as e:
        print(f"Error generating response: {e}")
else:
    print("\nOpenAI API key not set - skipping LLM response generation")

print_separator()

**Section 2: Templates for Complex Reasoning Chains**

In [ ]:
print("Section 2: Templates for Complex Reasoning Chains")

# Chain-of-Thought RAG Template
cot_rag_template = """
Analyze the provided information to answer the question.

CONTEXT:
{context}

QUESTION:
{question}

REASONING PROCESS:
1. Identify key facts and concepts from the context relevant to the question
2. Analyze relationships between these elements
3. Consider any constraints or conditions mentioned
4. Develop logical inferences step by step
5. Formulate a well-reasoned answer

Provide your complete reasoning process before giving the final answer.

ANSWER:
"""

# Tree of Thoughts Template
tot_template = """
Explore multiple reasoning paths to answer this question.

CONTEXT:
{context}

QUESTION:
{question}

SOLUTION APPROACH:
1. Path A: Consider first that...
   a. Analyze the implications...
   b. Evaluate whether this leads to...
   c. Determine if this approach is sufficient...

2. Path B: Alternatively, consider that...
   a. Analyze these implications...
   b. Evaluate whether this approach...
   c. Determine if this path is better...

3. Compare the outcomes of different reasoning paths
4. Select the most complete and accurate answer

Explicitly show your exploration of different reasoning approaches.

ANSWER:
"""

# Retrieval-Augmented Verification
verification_template = """
Answer the question and then verify your answer using the provided context.

CONTEXT:
{context}

QUESTION:
{question}

ANSWER PROCESS:
1. First, generate your best answer to the question
2. Then, analyze the context for specific information that confirms or contradicts your answer
3. Look for potential gaps or assumptions in your initial answer
4. Correct any errors or omissions based on the context
5. Provide your final, verified answer with appropriate confidence

FINAL ANSWER:
"""

# Test Chain-of-Thought template
example = reasoning_questions[0]
formatted_cot = cot_rag_template.format(
    context=example["context"],
    question=example["question"]
)

print("Chain-of-Thought Template Example:")
print(formatted_cot)
print(f"Tokens: {count_tokens(formatted_cot)}")

# Test Tree of Thoughts template
formatted_tot = tot_template.format(
    context=example["context"],
    question=example["question"]
)

print("\nTree of Thoughts Template Example:")
print(formatted_tot)
print(f"Tokens: {count_tokens(formatted_tot)}")

# Test Verification template
formatted_verification = verification_template.format(
    context=example["context"],
    question=example["question"]
)

print("\nVerification Template Example:")
print(formatted_verification)
print(f"Tokens: {count_tokens(formatted_verification)}")

# Compare reasoning templates
if os.environ.get("OPENAI_API_KEY"):
    try:
        print("\nComparing reasoning templates with LLM:")
        llm = ChatOpenAI(temperature=0)

        # Use a simpler question for demonstration
        simple_q = "Based on the context, why are some stars red and others blue?"
        simple_ctx = sample_docs[2].page_content

        # Standard template
        standard_template = """
        Answer the question based on the context.

        CONTEXT:
        {context}

        QUESTION:
        {question}

        ANSWER:
        """

        standard_prompt = standard_template.format(context=simple_ctx, question=simple_q)
        cot_prompt = cot_rag_template.format(context=simple_ctx, question=simple_q)

        standard_response = llm.invoke(standard_prompt).content
        cot_response = llm.invoke(cot_prompt).content

        print("\nStandard Template Response:")
        print(standard_response)

        print("\nChain-of-Thought Template Response:")
        print(cot_response)

    except Exception as e:
        print(f"Error comparing templates: {e}")
else:
    print("\nOpenAI API key not set - skipping template comparison")

print_separator()

**Section 3: Self-Reflective Prompts and Self-Correction**

In [ ]:
print("Section 3: Self-Reflective Prompts and Self-Correction")

# Self-Reflection Template
self_reflection_template = """
Answer the question using the provided information, then reflect on your answer.

CONTEXT:
{context}

QUESTION:
{question}

INITIAL ANSWER:
[Provide your best answer based on the context]

REFLECTION:
- What assumptions did I make in my answer?
- Did I use all relevant information from the context?
- Are there alternative interpretations I should consider?
- How confident am I in each part of my answer?
- What additional information would strengthen my answer?

IMPROVED ANSWER:
[Provide a refined answer based on your reflection]
"""

# Adversarial Self-Critique
adversarial_template = """
You will answer the question in two phases: first as an answer generator,
then as a critical reviewer looking for flaws.

CONTEXT:
{context}

QUESTION:
{question}

PHASE 1 - ANSWER:
[Generate your best answer based on the context]

PHASE 2 - CRITIQUE:
[Now adopt a skeptical perspective]
- Identify any unsupported claims in the answer
- Find any missing context or nuance
- Check if all relevant information was included
- Look for potential misinterpretations

FINAL ANSWER:
[Provide an improved answer addressing the critique]
"""

# Test Self-Reflection template
example = complex_data[0]
formatted_reflection = self_reflection_template.format(
    context=example["context"],
    question=example["question"]
)

print("Self-Reflection Template Example:")
print(formatted_reflection)
print(f"Tokens: {count_tokens(formatted_reflection)}")

# Test Adversarial Self-Critique template
formatted_adversarial = adversarial_template.format(
    context=example["context"],
    question=example["question"]
)

print("\nAdversarial Self-Critique Template Example:")
print(formatted_adversarial)
print(f"Tokens: {count_tokens(formatted_adversarial)}")

# Compare self-reflective templates
if os.environ.get("OPENAI_API_KEY"):
    try:
        print("\nComparing self-reflective templates with LLM:")
        llm = ChatOpenAI(temperature=0)

        adversarial_prompt = adversarial_template.format(
            context=example["context"],
            question=example["question"]
        )

        adversarial_response = llm.invoke(adversarial_prompt).content

        print("\nAdversarial Self-Critique Response:")
        print(adversarial_response)

    except Exception as e:
        print(f"Error comparing templates: {e}")
else:
    print("\nOpenAI API key not set - skipping template comparison")

print_separator()

**Section 4: Templates for Hybrid Human-AI Workflows**

In [ ]:
print("Section 4: Templates for Hybrid Human-AI Workflows")

# Uncertainty Highlighting Template
uncertainty_template = """
Answer the question based on the provided context.

CONTEXT:
{context}

QUESTION:
{question}

FORMAT YOUR ANSWER AS FOLLOWS:
1. Main answer with high-confidence statements
2. [UNCERTAIN: Areas where information is ambiguous or incomplete]
3. [MISSING: Key information needed to fully answer the question]
4. Suggested follow-up questions for clarification

ANSWER:
"""

# Feedback-Ready Template
feedback_template = """
Answer the question based on the context. Structure your response to facilitate expert feedback.

CONTEXT:
{context}

QUESTION:
{question}

ANSWER STRUCTURE:
1. Summary (key points in 1-2 sentences)
2. Detailed explanation with reasoning
   - Point A: [Evidence] → [Conclusion]
   - Point B: [Evidence] → [Conclusion]
3. Alternative interpretations if applicable
4. Confidence assessment (High/Medium/Low) with rationale
5. [FEEDBACK REQUESTED: Specific aspects where expert input would be valuable]

ANSWER:
"""

# Test Uncertainty Highlighting template
example = complex_data[1]
formatted_uncertainty = uncertainty_template.format(
    context=example["context"],
    question=example["question"]
)

print("Uncertainty Highlighting Template Example:")
print(formatted_uncertainty)
print(f"Tokens: {count_tokens(formatted_uncertainty)}")

# Test Feedback-Ready template
formatted_feedback = feedback_template.format(
    context=example["context"],
    question=example["question"]
)

print("\nFeedback-Ready Template Example:")
print(formatted_feedback)
print(f"Tokens: {count_tokens(formatted_feedback)}")

# Demonstrate hybrid workflow
if os.environ.get("OPENAI_API_KEY"):
    try:
        print("\nDemonstrating hybrid human-AI workflow:")
        llm = ChatOpenAI(temperature=0)

        # Initial response with uncertainty highlighting
        uncertainty_prompt = uncertainty_template.format(
            context=example["context"],
            question=example["question"]
        )

        initial_response = llm.invoke(uncertainty_prompt).content

        print("\nInitial Response with Uncertainty Highlighting:")
        print(initial_response)

        # Simulate human feedback
        human_feedback = """
        Expert feedback: The explanation about tidal heating is good, but you should mention
        that the gravitational interactions with other Jovian moons also contribute to Europa's
        heating. For Enceladus, the heating mechanism is indeed tidal, involving Saturn's gravity
        and orbital resonances with other moons, particularly Dione.
        """

        # Follow-up response incorporating feedback
        feedback_prompt = f"""
        You previously provided this answer:

        {initial_response}

        You've now received this expert feedback:

        {human_feedback}

        Please update your response to incorporate this feedback.
        """

        updated_response = llm.invoke(feedback_prompt).content

        print("\nUpdated Response After Human Feedback:")
        print(updated_response)

    except Exception as e:
        print(f"Error in human-AI workflow demo: {e}")
else:
    print("\nOpenAI API key not set - skipping human-AI workflow demo")

print_separator()

**Section 5: Comparative Analysis of Advanced Templates**

In [ ]:
print("Section 5: Comparative Analysis of Advanced Templates")

# Define template variations for comparison
template_variations = {
    "standard": """
    Answer the question based on the context.

    CONTEXT:
    {context}

    QUESTION:
    {question}

    ANSWER:
    """,

    "chain_of_thought": cot_rag_template,
    "self_reflection": self_reflection_template,
    "adversarial": adversarial_template,
    "uncertainty": uncertainty_template
}

# Define evaluation criteria
evaluation_criteria = [
    "Factual accuracy",
    "Reasoning depth",
    "Handling uncertainty",
    "Identifying assumptions",
    "Avoiding overconfidence",
    "Contextual relevance"
]

# Mock comparative analysis (would normally use actual LLM outputs)
# In a real scenario, these would be determined through human evaluation or automated metrics
comparative_scores = {
    "standard": [0.85, 0.45, 0.30, 0.25, 0.40, 0.80],
    "chain_of_thought": [0.88, 0.85, 0.55, 0.60, 0.65, 0.85],
    "self_reflection": [0.90, 0.80, 0.75, 0.85, 0.80, 0.85],
    "adversarial": [0.92, 0.75, 0.80, 0.90, 0.85, 0.85],
    "uncertainty": [0.85, 0.70, 0.90, 0.75, 0.90, 0.80]
}

# Display comparative analysis
print("Comparative Analysis of Template Effectiveness:")
print(f"{'Criteria':<25} | {'Standard':<10} | {'CoT':<10} | {'Self-Reflect':<10} | {'Adversarial':<10} | {'Uncertainty':<10}")
print("-" * 80)

for i, criterion in enumerate(evaluation_criteria):
    scores = [
        comparative_scores["standard"][i],
        comparative_scores["chain_of_thought"][i],
        comparative_scores["self_reflection"][i],
        comparative_scores["adversarial"][i],
        comparative_scores["uncertainty"][i]
    ]
    print(f"{criterion:<25} | {scores[0]:<10.2f} | {scores[1]:<10.2f} | {scores[2]:<10.2f} | {scores[3]:<10.2f} | {scores[4]:<10.2f}")

# Visualize the comparative analysis
try:
    # Create radar chart
    labels = evaluation_criteria
    angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
    angles += angles[:1]  # Close the polygon

    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(polar=True))

    for template_name, scores in comparative_scores.items():
        values = scores.copy()
        values += values[:1]  # Close the polygon
        ax.plot(angles, values, linewidth=2, label=template_name)
        ax.fill(angles, values, alpha=0.1)

    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_thetagrids(np.degrees(angles[:-1]), labels)
    ax.set_ylim(0, 1)
    ax.set_rlabel_position(0)
    ax.set_title("Advanced Template Comparison", fontsize=14, pad=20)
    ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Error creating visualization: {e}")
    print("To view the visualization, run this notebook in an environment that supports matplotlib.")

print("\nTemplate Strengths Analysis:")
print("- Standard templates: Good for straightforward factual questions")
print("- Chain-of-Thought: Excels at complex reasoning tasks requiring step-by-step analysis")
print("- Self-Reflection: Best for identifying assumptions and handling ambiguity")
print("- Adversarial: Strongest for avoiding factual errors and identifying gaps")
print("- Uncertainty Highlighting: Optimal for transparently communicating confidence levels")

print_separator()

**Section 6: Implementing Advanced Templates in Production**

In [ ]:
print("Section 6: Implementing Advanced Templates in Production")

print("Practical Implementation Considerations:")
print("\n1. Template Selection Strategy")
print("   Based on query characteristics, select the appropriate template:")
print("   - Factual questions → Standard templates")
print("   - Complex reasoning → Chain-of-Thought templates")
print("   - High-stakes decisions → Self-reflective or Adversarial templates")
print("   - Expert collaboration → Uncertainty or Feedback-Ready templates")
print("   - Multi-modal questions → Image-aware templates")

# Example template selector function
def select_advanced_template(query, context, has_images=False):
    """
    Select the most appropriate template based on query characteristics.
    """
    # Check if query has multi-modal elements
    if has_images:
        return multimodal_template

    # Check if query appears to require complex reasoning
    reasoning_indicators = ["why", "how", "explain", "reason", "analyze",
                            "compare", "contrast", "evaluate", "what if"]
    if any(indicator in query.lower() for indicator in reasoning_indicators):
        return cot_rag_template

    # Check if query might benefit from uncertainty handling
    uncertainty_indicators = ["possible", "might", "could", "uncertain",
                              "likelihood", "probability", "estimate"]
    if any(indicator in query.lower() for indicator in uncertainty_indicators):
        return uncertainty_template

    # Check if query is sensitive or high-stakes
    sensitive_topics = ["health", "medical", "legal", "financial",
                         "safety", "critical", "important decision"]
    if any(topic in query.lower() or topic in context.lower() for topic in sensitive_topics):
        return adversarial_template

    # Default to standard template
    return template_variations["standard"]

# Test template selection
test_queries = [
    "What is the diameter of the Milky Way galaxy?",
    "Why might supermassive black holes affect star formation in galaxies?",
    "What is the likelihood of finding habitable exoplanets around red dwarf stars?",
    "What medical conditions might arise from long-term space travel?",
    "Compare the star formation process in different nebula types."
]

print("\n2. Template Selection Examples:")
for query in test_queries:
    template = select_advanced_template(query, sample_docs[0].page_content)
    template_name = "Standard"
    for name, tmpl in template_variations.items():
        if template == tmpl:
            template_name = name
    print(f"Query: '{query}'")
    print(f"Selected template: {template_name}")
    print()

print("\n3. Token Usage Management")
print("   Advanced templates consume more tokens but may provide better results.")
print("   Token usage comparison:")

# Compare token usage
test_context = sample_docs[0].page_content
test_question = "Explain how the position of our solar system in the Milky Way affects our astronomical observations."

token_usage = {}
for name, template in template_variations.items():
    formatted = template.format(context=test_context, question=test_question)
    token_count = count_tokens(formatted)
    token_usage[name] = token_count

# Sort by token usage
sorted_usage = sorted(token_usage.items(), key=lambda x: x[1])
for name, count in sorted_usage:
    print(f"   - {name}: {count} tokens")

print("\n4. Template Chaining Strategy")
print("   For complex RAG applications, chain multiple templates:")
print("   - First use Chain-of-Thought to generate an initial answer")
print("   - Then use Self-Reflection or Adversarial to refine the answer")
print("   - Finally use Uncertainty Highlighting for presentation")

print("\n5. Monitoring and Evaluation")
print("   In production, continuously monitor template performance:")
print("   - Track factual accuracy, reasoning quality, and user satisfaction")
print("   - A/B test template variations on similar queries")
print("   - Collect human feedback on different template outputs")
print("   - Adjust template selection strategy based on performance data")

print_separator()

**Section 7: Case Studies and Real-World Applications**

In [ ]:
print("Section 7: Case Studies and Real-World Applications")

print("Case Study 1: Scientific Research Assistant")
print("\nChallenge: Researchers need accurate information from scientific papers with proper source attribution and uncertainty communication.")
print("\nSolution: Chain-of-Thought + Uncertainty Highlighting template")

scientific_template = """
Analyze the scientific information to answer the research question.

SOURCES:
{context}

RESEARCH QUESTION:
{question}

ANALYSIS APPROACH:
1. Identify relevant findings and methodologies from the sources
2. Evaluate the strength of evidence for each finding
3. Analyze any conflicting results or interpretations
4. Consider methodological limitations that affect conclusions
5. Synthesize a comprehensive answer

FORMAT:
- Main findings: [High confidence statements with citations]
- Interpretations: Step-by-step reasoning connecting findings to conclusions
- Limitations: [UNCERTAINTY: Areas where evidence is limited or conflicting]
- Research gaps: [MISSING: Important information not addressed in the sources]
- Future directions: Suggestions for further investigation

ANSWER:
"""

print("Example scientific research template application:")
print("Query: 'What is the current consensus on exoplanet habitability indicators?'")
print("Uses Chain-of-Thought reasoning to analyze conflicting research findings")
print("Highlights uncertainty in specific areas where evidence is limited")
print("Explicitly notes information gaps to guide further research")

print("\nCase Study 2: Medical Decision Support")
print("\nChallenge: Clinicians need accurate medical information with clear indication of confidence levels and potential alternatives.")
print("\nSolution: Adversarial Self-Critique + Feedback-Ready template")

medical_template = """
Analyze the medical information to address the clinical question.

MEDICAL SOURCES:
{context}

CLINICAL QUESTION:
{question}

TWO-PHASE ANALYSIS:

PHASE 1 - INITIAL ASSESSMENT:
1. Relevant clinical findings from the sources
2. Evidence-based interpretations
3. Potential diagnostic or treatment considerations

PHASE 2 - CRITICAL REVIEW:
1. Identify evidence limitations or contradictions
2. Consider alternative interpretations
3. Assess generalizability to the specific clinical scenario
4. Identify potential risks or contraindications

FORMAT:
- Summary: Brief overview of key points
- Evidence: [Source X] Specific findings → Clinical implications
- Confidence level: (High/Moderate/Low) with explanation
- Alternatives: Important differential considerations
- [CONSULTATION RECOMMENDED]: Specific aspects requiring clinical judgment

RESPONSE:
"""

print("Example medical decision support template application:")
print("Query: 'What are the latest treatment approaches for resistant hypertension?'")
print("Uses adversarial approach to critically evaluate treatment options")
print("Explicitly labels confidence levels for different recommendations")
print("Highlights areas where physician judgment is particularly important")

print("\nCase Study 3: Legal Research Assistant")
print("\nChallenge: Legal professionals need accurate citation of relevant statutes and precedents with nuanced interpretation.")
print("\nSolution: Tree of Thoughts + Attribution template")

legal_template = """
Analyze the legal information to address the question.

LEGAL SOURCES:
{context}

LEGAL QUESTION:
{question}

ANALYSIS PATHS:

PATH A - STATUTORY INTERPRETATION:
1. Identify relevant statutes and regulations
2. Analyze the plain language meaning
3. Consider legislative intent if available
4. Evaluate applicability to the specific scenario

PATH B - CASE LAW ANALYSIS:
1. Identify relevant precedents
2. Extract key holdings and principles
3. Analyze factual similarities and differences
4. Consider jurisdictional limitations

PATH C - DOCTRINAL APPROACH:
1. Identify legal doctrines at issue
2. Analyze how courts have applied these doctrines
3. Consider scholarly interpretations
4. Evaluate evolution of doctrine over time

FORMAT:
- Summary conclusion based on most applicable analysis path
- Citations in proper legal format [Case/Statute X]
- Explicit reasoning connecting authorities to conclusion
- Alternative interpretations where authorities conflict
- Limitations: Jurisdictional constraints or distinguishing factors

LEGAL ANALYSIS:
"""

print("Example legal research template application:")
print("Query: 'What are the requirements for establishing fair use in copyright infringement cases?'")
print("Explores multiple analytical approaches (statutory, case law, and doctrinal)")
print("Provides precise legal citations in proper format")
print("Acknowledges jurisdictional limitations and conflicting precedents")

print("\nCase Study 4: Educational Content Creation")
print("\nChallenge: Educators need accurate explanations at appropriate levels with clear reasoning and learning scaffolds.")
print("\nSolution: Chain-of-Thought + Multimodal template")

educational_template = """
Create educational content to answer the student's question.

SUBJECT MATERIAL:
{context}

STUDENT QUESTION:
{question}

EDUCATIONAL LEVEL:
{educational_level}

LEARNING APPROACH:
1. Start with core concepts at the appropriate educational level
2. Provide a clear step-by-step explanation of the reasoning
3. Connect to visual references where applicable
4. Include analogies or examples to illustrate abstract concepts
5. Anticipate common misconceptions and address them

FORMAT YOUR RESPONSE:
- Core Concept: Brief explanation of the fundamental idea
- Step-by-Step Reasoning: Clear logical progression
- Visual References: Refer to [Image X] where helpful
- Practical Example: Concrete application of the concept
- Check Understanding: Simple question to verify comprehension

EDUCATIONAL RESPONSE:
"""

print("Example educational template application:")
print("Query: 'How do black holes affect spacetime?'")
print("Educational level: High school physics")
print("Provides step-by-step reasoning with appropriate analogies")
print("References relevant diagrams and visual representations")
print("Anticipates and addresses common misconceptions")

print_separator()

**Section 8: Building a Comprehensive Advanced Template System**

In [ ]:
print("Section 8: Building a Comprehensive Advanced Template System")

class AdvancedTemplateSystem:
    """A complete system for managing and deploying advanced RAG templates."""

    def __init__(self):
        # Initialize template library
        self.templates = {
            "standard": template_variations["standard"],
            "chain_of_thought": cot_rag_template,
            "tree_of_thoughts": tot_template,
            "verification": verification_template,
            "self_reflection": self_reflection_template,
            "adversarial": adversarial_template,
            "multimodal": multimodal_template,
            "uncertainty": uncertainty_template,
            "feedback_ready": feedback_template,
            "scientific": scientific_template,
            "medical": medical_template,
            "legal": legal_template,
            "educational": educational_template
        }

        # Template categories for organization
        self.categories = {
            "reasoning": ["chain_of_thought", "tree_of_thoughts", "verification"],
            "reflective": ["self_reflection", "adversarial"],
            "domain_specific": ["scientific", "medical", "legal", "educational"],
            "interactive": ["feedback_ready", "uncertainty"],
            "multimodal": ["multimodal"]
        }

    def get_template(self, name):
        """Get a template by name."""
        return self.templates.get(name, self.templates["standard"])

    def list_templates(self, category=None):
        """List available templates, optionally filtered by category."""
        if category:
            return [name for name in self.templates if name in self.categories.get(category, [])]
        return list(self.templates.keys())

    def analyze_query(self, query, context=None):
        """Analyze query characteristics to recommend template."""
        features = {
            "reasoning_required": False,
            "uncertainty_present": False,
            "domain_specific": None,
            "multimodal_elements": False,
            "complexity_level": "low"
        }

        # Check for reasoning indicators
        reasoning_terms = ["why", "how", "explain", "reason", "analyze", "compare", "evaluate"]
        if any(term in query.lower() for term in reasoning_terms):
            features["reasoning_required"] = True
            features["complexity_level"] = "medium"

        # Check for uncertainty indicators
        uncertainty_terms = ["possible", "might", "could", "uncertain", "probability"]
        if any(term in query.lower() for term in uncertainty_terms):
            features["uncertainty_present"] = True

        # Check for domain indicators
        domains = {
            "scientific": ["research", "study", "experiment", "hypothesis", "theory", "scientific"],
            "medical": ["patient", "treatment", "diagnosis", "medical", "clinical", "symptoms"],
            "legal": ["law", "legal", "regulation", "statute", "court", "case", "precedent"],
            "educational": ["learn", "teach", "student", "education", "concept", "understand"]
        }

        for domain, indicators in domains.items():
            if any(term in query.lower() for term in indicators):
                features["domain_specific"] = domain
                break

        # Check for multimodal indicators
        multimodal_terms = ["image", "picture", "diagram", "graph", "figure", "visual"]
        if any(term in query.lower() for term in multimodal_terms):
            features["multimodal_elements"] = True

        # Assess complexity
        if len(query.split()) > 15 or "," in query or "and" in query:
            features["complexity_level"] = "high"

        return features

    def recommend_template(self, query, context=None, has_images=False):
        """Recommend the most appropriate template based on query analysis."""
        features = self.analyze_query(query, context)

        # Check for domain-specific templates first
        if features["domain_specific"]:
            domain_template = features["domain_specific"]
            if domain_template in self.templates:
                return domain_template

        # Check for multimodal needs
        if features["multimodal_elements"] or has_images:
            return "multimodal"

        # High complexity queries
        if features["complexity_level"] == "high":
            if features["reasoning_required"]:
                return "tree_of_thoughts"
            else:
                return "verification"

        # Medium complexity queries
        if features["complexity_level"] == "medium":
            if features["reasoning_required"]:
                return "chain_of_thought"
            elif features["uncertainty_present"]:
                return "uncertainty"
            else:
                return "self_reflection"

        # Default to standard for simple queries
        return "standard"

    def format_prompt(self, template_name, **kwargs):
        """Format a prompt using the specified template and variables."""
        template = self.get_template(template_name)
        formatted = template.format(**kwargs)

        # Print token usage for monitoring
        token_count = count_tokens(formatted)
        print(f"Using template '{template_name}' ({token_count} tokens)")

        return formatted

    def generate_response(self, query, context, llm=None, template_name=None, **kwargs):
        """Generate a response using the appropriate template."""
        # If no template specified, recommend one
        if not template_name:
            template_name = self.recommend_template(query, context)

        # Format the prompt
        prompt = self.format_prompt(
            template_name,
            context=context,
            question=query,
            **kwargs
        )

        # Generate response if LLM provided
        if llm:
            try:
                response = llm.invoke(prompt)
                return {
                    "template_used": template_name,
                    "prompt": prompt,
                    "response": response.content,
                    "token_count": count_tokens(prompt)
                }
            except Exception as e:
                return {
                    "error": str(e),
                    "template_used": template_name,
                    "prompt": prompt
                }
        else:
            # Return the formatted prompt if no LLM
            return {
                "template_used": template_name,
                "prompt": prompt,
                "token_count": count_tokens(prompt)
            }

# Create the template system
template_system = AdvancedTemplateSystem()

# Demonstrate template recommendations
print("Template Recommendation Examples:")
test_queries = [
    "What is the diameter of the Milky Way galaxy?",
    "Why might supermassive black holes affect star formation in galaxies?",
    "What is the likelihood of finding habitable exoplanets around red dwarf stars?",
    "What treatments are recommended for patients with resistant hypertension?",
    "Can you explain how black holes bend spacetime using the diagram?",
    "What legal precedents establish the fair use doctrine in copyright law?"
]

for query in test_queries:
    recommended = template_system.recommend_template(query)
    features = template_system.analyze_query(query)
    print(f"Query: '{query}'")
    print(f"Features: {features}")
    print(f"Recommended template: {recommended}")
    print()

# Demonstrate template usage
example_query = "How do stars form and why do they have different colors?"
example_context = sample_docs[2].page_content

result = template_system.generate_response(
    example_query,
    example_context,
    template_name="chain_of_thought"
)

print("\nGenerated Chain-of-Thought Prompt:")
print(result["prompt"])
print(f"Token count: {result['token_count']}")

# Demonstrate with LLM if available
if os.environ.get("OPENAI_API_KEY"):
    try:
        print("\nGenerating response with LLM:")
        llm = ChatOpenAI(temperature=0)

        result_with_llm = template_system.generate_response(
            example_query,
            example_context,
            llm=llm
        )

        print(f"Template used: {result_with_llm['template_used']}")
        print(f"Response: {result_with_llm['response']}")

    except Exception as e:
        print(f"Error generating LLM response: {e}")
else:
    print("\nOpenAI API key not set - skipping LLM response generation")

print_separator()

print("Building an advanced template system provides:")
print("1. Consistency across different query types and domains")
print("2. Automatic selection of appropriate templates")
print("3. Centralized management of template variations")
print("4. Easier monitoring of token usage and performance")
print("5. Simplified integration with RAG pipelines")

print_separator()

**Section 9: Best Practices and Future Directions**

In [ ]:
print("Section 9: Best Practices and Future Directions")

print("Best Practices for Advanced Templates:")
print("\n1. Template Documentation")
print("   - Maintain clear documentation for each template")
print("   - Record the purpose, strengths, and limitations")
print("   - Document token usage patterns and optimization notes")
print("   - Include example queries where the template excels")

print("\n2. Performance Monitoring")
print("   - Track template performance across different query types")
print("   - Monitor token usage, response time, and quality metrics")
print("   - Collect user feedback on response quality")
print("   - Regularly update templates based on performance data")

print("\n3. Template Versioning")
print("   - Maintain version history for all templates")
print("   - Document changes between versions")
print("   - Test new versions before deployment")
print("   - Support rollback to previous versions if needed")

print("\n4. Specialized Template Development")
print("   - Create domain-specific templates for key use cases")
print("   - Customize templates for different user personas")
print("   - Develop templates optimized for different context lengths")
print("   - Build templates for specific reasoning patterns")

print("\n5. Hybrid Approaches")
print("   - Use template chaining for complex queries")
print("   - Combine automatic and manual template selection")
print("   - Integrate human feedback loops for continuous improvement")
print("   - Develop templates that facilitate human-AI collaboration")

print("\nFuture Directions in Advanced Prompting:")

print("\n1. Dynamic Template Generation")
print("   - AI-generated templates customized for specific queries")
print("   - Templates that adapt based on user interaction patterns")
print("   - Self-optimizing templates that evolve based on performance")

print("\n2. Multi-step Prompting Frameworks")
print("   - Frameworks for breaking complex tasks into multiple prompts")
print("   - Pipelines that combine different template types")
print("   - Meta-prompts that guide the selection of sub-prompts")

print("\n3. Personalized Templates")
print("   - Templates adapted to user expertise levels")
print("   - Customization based on user preferences")
print("   - Domain-specific variations for different industries")

print("\n4. Enhanced Multi-modal Integration")
print("   - Templates for integrating multiple data modalities")
print("   - Vision-language-retrieval coordination templates")
print("   - Templates for generating multimodal outputs")

print("\n5. Reasoning-Enhanced Templates")
print("   - Templates with deeper cognitive scaffolding")
print("   - Built-in logical validation mechanisms")
print("   - Templates that guide formal reasoning patterns")

print_separator()

print("Notebook completed!")

# Key Takeaways:
# 1. Advanced templates enable more sophisticated RAG capabilities like multi-modal integration and complex reasoning
# 2. Self-reflective prompts and adversarial approaches can significantly improve response quality
# 3. Domain-specific templates provide superior performance for specialized applications
# 4. Building a comprehensive template system simplifies management and optimization
# 5. Future developments will likely focus on dynamic, personalized, and multi-step prompting frameworks